<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-modules" data-toc-modified-id="Import-modules-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import modules</a></span></li><li><span><a href="#RequestFunc" data-toc-modified-id="RequestFunc-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>RequestFunc</a></span></li><li><span><a href="#Main" data-toc-modified-id="Main-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Main</a></span></li></ul></div>

# Import modules 

In [1]:
import requests
import pandas as pd
from copy import deepcopy
from datetime import datetime
from time import sleep
import os

parent = os.path.dirname(os.path.abspath("__file__"))
output_path = os.path.join(parent, 'Opt_OI')
if not os.path.isdir(output_path):
    os.makedirs(output_path)

# RequestFunc

In [2]:
def GetOptionOI(ticker="QQQ"):
    
    #=====================
    # Read Update Data
    #=====================
    url = f"https://cdn.cboe.com/api/global/delayed_quotes/options/{ticker.upper()}.json"
    res = requests.get(url)
    data = res.json()
    opts_ = data['data']['options']

    #=====================
    # Setup Output Path
    #=====================
    trade_date = pd.to_datetime(data['data']['last_trade_time']) # datetime.today()
    output_ = os.path.join(output_path, str(trade_date.year), str(trade_date.month).zfill(2), str(trade_date.day).zfill(2))
    if not os.path.isdir(output_):
        os.makedirs(output_)
    #------------------------
    # Take Update Option Info
    #------------------------
    updated_datas = []
    for opt in opts_:
        opt_symbol = opt['option']
        underlying = opt_symbol[:-15]
        strike = int(opt_symbol[-8:]) / 1000
        c_p = opt_symbol[-9:-8]
        ttm = opt_symbol[-15:-9]
#         print(underlying, ttm, c_p, strike)
        opt.update({
            'underlying':underlying,
            'ttm':ttm,
            'strike':strike,
            'c_p':c_p
        })
        updated_datas.append(opt)

    #========================
    # Aggregate wit DataFrame
    #========================
    opt_df = pd.DataFrame(updated_datas)
    outputRawOI(opt_df, output_, ticker)
    nearby_df = opt_df[opt_df['ttm'].apply(lambda x: x[:4]) == opt_df['ttm'].apply(lambda x: x[:4]).unique()[0]]

    #------------------------
    # Seperate Call and Put
    #------------------------
    filted_ttm_g = {"C":[], "P":[]}
    ttm_group = nearby_df.groupby('ttm')
    for ttm in ttm_group.groups:
        print(ttm)
        ttm_g = ttm_group.get_group(ttm)
        ttm_g = ttm_g['option,underlying,strike,ttm,c_p,open_interest'.split(',')]
        for C_P in ttm_g.c_p.unique():
            filted_ttm_g[C_P].append(ttm_g[ttm_g.c_p == C_P].set_index('strike'))

    #-------------------------
    # Create dataframe of call
    #-------------------------
    call_df = pd.DataFrame()
    call_df.index.name = 'strike'
    for c_df in filted_ttm_g["C"]:
        if call_df.empty:
            call_df = c_df
        else:
            for row in c_df.itertuples():
                try:
                    call_df.loc[row.Index, 'open_interest'] += row.open_interest
                except:
                    tmp_row = dict(row._asdict())
                    tmp_row['strike'] = tmp_row['Index']
                    del tmp_row['Index']
                    call_df = call_df.append(pd.DataFrame([tmp_row]).set_index('strike'))
                    
    #-------------------------
    # Create dataframe of put
    #-------------------------
    put_df = pd.DataFrame()
    put_df.index.name = 'strike'
    for p_df in filted_ttm_g["P"]:
        if put_df.empty:
            put_df = p_df
        else:
            for row in p_df.itertuples():
                try:
                    put_df.loc[row.Index, 'open_interest'] += row.open_interest
                except:
                    tmp_row = dict(row._asdict())
                    tmp_row['strike'] = tmp_row['Index']
                    del tmp_row['Index']
                    put_df = put_df.append(pd.DataFrame([tmp_row]).set_index('strike'))
    
    #========================
    # Save Today Info
    #========================
    tmp_call_df = call_df.sort_values('open_interest', ascending=False).head(10).reset_index()['strike,open_interest'.split(',')]
    tmp_call_df['oi_diff'] = 0

    tmp_put_df = put_df.sort_values('open_interest', ascending=False).head(10).reset_index()['strike,open_interest'.split(',')]
    tmp_put_df['oi_diff'] = 0

    
    with pd.ExcelWriter(os.path.join(output_, f"{ticker}_Option_OI.xlsx")) as writer:
        tmp_call_df.to_excel(writer, encoding='utf-8-sig', index=False, sheet_name="CALL")
        tmp_put_df.to_excel(writer, encoding='utf-8-sig', index=False, sheet_name="PUT")
        
    sleep(3)

In [3]:
def outputRawOI(opt_df, output_, ticker):
    with pd.ExcelWriter(os.path.join(output_, f"{ticker}_Option_OI_raw.xlsx")) as writer:
        ttms = sorted(opt_df.ttm.unique())
        for ttm in ttms:
            ttm_df = opt_df[opt_df.ttm == ttm]
            tmp_df_c = ttm_df[ttm_df.c_p == "C"]
            tmp_df_p = ttm_df[ttm_df.c_p == "P"]
    
            tmp_df_c.to_excel(writer, encoding='utf-8-sig', index=False, sheet_name=ttm)
            tmp_df_p.to_excel(writer, encoding='utf-8-sig', index=False, sheet_name=ttm, startcol = 29)

# Main

In [4]:
for ticker in 'QQQ,TQQQ,ARKK,SMH'.split(','): # used to follow NASDAQ
    GetOptionOI(ticker)
    break

for ticker in 'SPY,IWM'.split(','): # used to follow SPY
    GetOptionOI(ticker)
    
for ticker in 'TSLA,BITO,COIN'.split(','): # used to follow TLSA
    GetOptionOI(ticker)
    
for ticker in 'TSM,HNHPF,ASX,SIMO,HIMX,UMC,GIGM,CHT,IMOS,LEDS,APWC,GRMN'.split(','): # used to follow Taiwan Stock ADR
    try:
        GetOptionOI(ticker)    
    except:
        print(ticker, "Failed")

for ticker in 'BABA,JD,BIDU'.split(','): # used to follow BABA
    try:
        GetOptionOI(ticker)    
    except:
        print(ticker, "Failed")

for ticker in 'GLD'.split(','): # used to follow XAUUSD 黃金現貨
    try:
        GetOptionOI(ticker)    
    except:
        print(ticker, "Failed")


220725
220727
220729
220725
220727
220729
220725
220727
220729
220729
220729
220729
220729
HNHPF Failed
220819
220819
220729
220819
GIGM Failed
220819
220819
LEDS Failed
APWC Failed
220819
220729
220729
220729
220729
